## Realistic features - to scale, in target coordinates

In [1]:
# import packages
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import ticker as mtick
from scipy import special

# switch on inline plotting
%matplotlib inline

# append path for loading own packages
import sys
sys.path.append("../packages")

# import own packages
import edges
import grating
import copy
from auxiliary import theta
from auxiliary import evp
import auxiliary as aux
import jitter
import density

In [2]:
def transpose_extent(a):
    return [a[2], a[3], a[0], a[1]]

In [7]:
def run(**params):
    """
    Calculate a 2D density distribution and possibly write it and other output for validation to file.
    
    Arguments:
    
    
    """
    parameter_string = \
        "{:03.0f}".format(evp(params['edge_width'])       *1e9) + evp(params['edge_type']) + "_" +\
        "{:03.0f}".format(evp(params['pitch'])            *1e9) + "pitch_" +\
        "{:03.0f}".format(evp(params['jitter_amplitude']) *1e9) + "jampl_" +\
        "{:03.0f}".format(evp(params['jitter_wavelength'])*1e9) + "jwl"
    
    outfilename = parameter_string + "_summeddensity.txt"
    print(parameter_string)
    
    # Make density function
    feature          = grating.symm_feat(       evp(params['hfs']),
                                                evp(params['edge_function']))
    
    grating_function = grating.feature_grating( evp(params['pitch']),
                                                feature)
    
    jitter_function  = jitter.make_jitter(      evp(params['jitter_amplitude']),
                                                evp(params['jitter_wavelength']))
    
    density_function = density.make_density(    grating_function,
                                                jitter_function)
    
    # Plot functions for validation
    if evp(params['validation_plot']) == True:
        fig = plt.figure(figsize=(20,20))
        
        sub1 = fig.add_subplot(3, 3, 1, adjustable='datalim', aspect=1.)
        plt.plot(evp(params['yp']), evp(params['edge_function'])(evp(params['yp'])))
        plt.title('Edge function')
        plt.xlabel('Target y coordinate [m]')
        plt.ylabel('Target x coordinate [m]')
        sub1.get_xaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        sub1.get_yaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        
        sub2 = fig.add_subplot(3, 2, 2, adjustable='datalim', aspect=1.)
        plt.plot(evp(params['yp']), feature(evp(params['yp'])))
        plt.title('Feature function')
        plt.xlabel('Target y coordinate [m]')
        plt.ylabel('Target x coordinate [m]')
        sub2.get_xaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        sub2.get_yaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        
        sub3 = fig.add_subplot(3, 2, 3, adjustable='datalim', aspect=1.)
        plt.plot(evp(params['yp']), grating_function(evp(params['yp'])))
        plt.title('Grating function')
        plt.xlabel('Target y coordinate [m]')
        plt.ylabel('Target x coordinate [m]')
        sub3.get_xaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        sub3.get_yaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        
        sub4 = fig.add_subplot(3, 2, 4, adjustable='datalim', aspect=1.)
        plt.plot(jitter_function(evp(params['zp'])), evp(params['zp']))
        plt.title('Jitter function')
        plt.xlabel('Target y coordinate [m]')
        plt.ylabel('Target z coordinate [m]')
        sub4.get_xaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        sub4.get_yaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        
        path = parameter_string + "_validate.pdf"
        if not aux.path_exists(path, mode='Fail'):
            fig.savefig(path)
        plt.close(fig)
    
    # Calculate and sum density slices along z axis
    xx, yy = np.meshgrid(evp(params['x']), evp(params['y']))
    
    def dens2D (xx, yy, z):
        dd = np.zeros(shape=xx.shape)
        for this_z in z:
            xxp, yyp, zp = aux.rcoords2tcoords(xx, yy, this_z)
            dd += density_function(xxp, yyp, zp)
        return evp(params['nelectrons']) * dd
    
    dd_sum = dens2D(xx, yy, evp(params['z']))
    
    # Save summed density to file
    if evp(params['save_summed_density']) == True:
        if not aux.path_exists(outfilename, mode='Fail'):
            np.savetxt(outfilename, dd_sum)
    
    # Plot summed density
    if evp(params['summed_density_plot']) == True:
        fig = plt.figure(figsize=(20, 6))
        sub1 = fig.add_subplot(1, 1, 1)
        
        aux.imshow(dd_sum.T, extent=transpose_extent(evp(params['extent_laser_xy'])))
        sub1.get_xaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        sub1.get_yaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        plt.title('xy plot summed slices')
        plt.colorbar()
        
        path = parameter_string + "_summeddensity_plot.pdf"
        if not aux.path_exists(path, mode='Fail'):
            fig.savefig(path)
        plt.close(fig)
    
    # 3D density grid
    if evp(params['calc3d']):
        # Calculate 3D density grid
        xxx, yyy, zzz = np.meshgrid(evp(params['x']), evp(params['y']), evp(params['z']))
        xxxp, yyyp, zzzp = aux.rcoords2tcoords(xxx, yyy, zzz)
        %time ddd = evp(params['nelectrons']) * density_function(xxxp, yyyp, zzzp)
        
        # Plot silces of 3D density
        fig = plt.figure(figsize=(20, 6))
        
        sub1 = fig.add_subplot(1, 3, 1)
        aux.imshow(ddd[:, :, len(z_test)//8].T, extent = transpose_extent(evp(params['extent_laser_xy'])), origin='upper')
        sub1.get_xaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        sub1.get_yaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        plt.title('xy plot at 1/8 of z range')
        plt.colorbar()
    
        sub2 = fig.add_subplot(1, 3, 2)
        aux.imshow(ddd[:, :, len(z_test)//2].T, extent = transpose_extent(evp(params['extent_laser_xy'])), origin='upper')
        sub2.get_xaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        sub2.get_yaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        plt.title('xy plot at 1/2 of z range')
        plt.colorbar()
        
        sub3 = fig.add_subplot(1, 3, 3)
        aux.imshow(ddd[:, :, len(z_test)//8*7].T, extent = transpose_extent(evp(params['extent_laser_xy'])), origin='upper')
        sub3.get_xaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        sub3.get_yaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
        plt.title('xy plot at 7/8 of z range')
        plt.colorbar()

### Measures / parameter set

**pitch**: Horizontal feature pattern length (50 nm, 100 nm, 400 nm)

**hfs**: Horizontal feature size (measured half height to half height, always 50 % of *pitch*)

**vfs**: Vertical feature size (measured from asymptotic bootom to asymptotic top, same as *hfs*)

In [17]:
#pitches = [50e-9, 100e-9, 400e-9]
pitches = [400e-9]

#jitter_wavelengths = [5e-9, 15e-9, 50e-9]
jitter_wavelengths = [50e-9]

In [2]:
params = {}

params['hfs'] = lambda : 0.5 * evp(params['pitch'])
params['vfs'] = lambda : 0.5 * evp(params['pitch'])

# Amplitude of jitter in m
params['jitter_amplitude'] = 15e-9

# Width of one feature edge in m
params['edge_width'] = 20e-9

# 3D density of silicon in electrons per m^3
dens3_silicon = 7e29

# 3D density of grating in electrons per m^3
params['dens3_grating'] = dens3_silicon

# 3D density of foil (without grating) in electrons per m^3
params['dens3_foil'] = dens3_silicon

# Focus size (edge length) in m
params['focus_size'] = 2e-6

# Number of incoming photons
params['Nin'] = 10e10

# Number of electrons in a cell of full density
params['nelectrons'] = lambda : (evp(params['xmax']) - evp(params['xmin'])) / evp(params['xres']) *\
                                 (evp(params['ymax']) - evp(params['ymin'])) / evp(params['yres']) *\
                                 (evp(params['zmax']) - evp(params['zmin'])) / evp(params['zres']) *\
                                  evp(params['dens3_foil'])

# Define laser coordinate range and resolution
params['xres'] = 512
params['xmin'] = -0.5 * params['focus_size']
params['xmax'] =  0.5 * params['focus_size']

params['yres'] = 512
params['ymin'] = -0.5 * params['focus_size']
params['ymax'] =  0.5 * params['focus_size']

params['zres'] = 512
params['zmin'] = lambda : -0.5 * evp(params['focus_size']) - np.sqrt(2)*evp(params['vfs'])
params['zmax'] = lambda :  0.5 * evp(params['focus_size'])

# Calculate target coordinate range
params['xpmin'] = lambda : aux.rcoords2tcoords(evp(params['xmax']), 0, 0)[0]
params['xpmax'] = lambda : aux.rcoords2tcoords(evp(params['xmin']), 0, 0)[0]
params['ypmin'] = lambda : aux.rcoords2tcoords(0, evp(params['ymin']), 0)[1]
params['ypmax'] = lambda : aux.rcoords2tcoords(0, evp(params['ymax']), 0)[1]
params['zpmin'] = lambda : aux.rcoords2tcoords(0, 0, evp(params['zmax']))[2]
params['zpmax'] = lambda : aux.rcoords2tcoords(0, 0, evp(params['zmin']))[2]

# Define edge functions
edgess = [(lambda : edges.make_erf_edge(   evp(params['edge_width']), evp(params['vfs'])), 'erfedge'),
          (lambda : edges.make_skewed_edge(evp(params['edge_width']), evp(params['vfs'])), 'skewededge'),
          (lambda : edges.make_perf_edge(  evp(params['edge_width']), evp(params['vfs'])), 'perfedge')]

# Define behavior
params['calc3d'] = False
params['only_count'] = False
params['validation_plot'] = True
params['save_summed_density'] = True
params['summed_density_plot'] = True

In [20]:
# Run parameter scan
count = 0
for pitch in pitches:
    params['pitch'] = pitch
    
    # Laser coordinate extents (for plotting)
    params['extent_laser_xy'] = [evp(params['xmin']), evp(params['xmax']),\
                                 evp(params['ymin']), evp(params['ymax'])]
    params['extent_laser_xz'] = [evp(params['xmin']), evp(params['xmax']),\
                                 evp(params['zmin']), evp(params['zmax'])]
    params['extent_laser_yz'] = [evp(params['ymin']), evp(params['ymax']),\
                                 evp(params['zmin']), evp(params['zmax'])]
    
    # Make 1D laser coordinate arrays
    params['x'] = np.linspace(evp(params['xmin']), evp(params['xmax']), evp(params['xres']))
    params['y'] = np.linspace(evp(params['ymin']), evp(params['ymax']), evp(params['yres']))
    params['z'] = np.linspace(evp(params['zmin']), evp(params['zmax']), evp(params['zres']))
    
    
    # Target coordinate extents (for plotting)
    params['extent_target_xy'] = [evp(params['xpmin']), evp(params['xpmin']),\
                                  evp(params['ypmin']), evp(params['ypmin'])]
    params['extent_target_xz'] = [evp(params['xpmin']), evp(params['xpmin']),\
                                  evp(params['zpmin']), evp(params['zpmin'])]
    params['extent_target_yz'] = [evp(params['ypmin']), evp(params['ypmin']),\
                                  evp(params['zpmin']), evp(params['zpmin'])]
    
    # Make 1D target coordinate arrays
    params['xp'] = np.linspace(evp(params['xpmin']), evp(params['xpmax']), evp(params['xres']))
    params['yp'] = np.linspace(evp(params['ypmin']), evp(params['ypmax']), evp(params['yres']))
    params['zp'] = np.linspace(evp(params['zpmin']), evp(params['zpmax']), evp(params['zres']))
    
    for jitter_wavelength in jitter_wavelengths:
        params['jitter_wavelength'] = jitter_wavelength
        
        for edge in edgess:
            params['edge_function'] = edge[0]
            params['edge_type']     = evp(edge[1])
            
            count += 1
            print(count)
            if not params['only_count']:
                run(**params)
            
            print("")

print("Number of cases: {}".format(count))

1
020erfedge_400pitch_015jampl_050jwl

2
020skewededge_400pitch_015jampl_050jwl

3
020perfedge_400pitch_015jampl_050jwl

Number of cases: 3


Strange behavior - x and y axes seem swapped in the 3D array (?!)